In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-26 09:36:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-08-26 09:36:11 (88.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [5]:
print(f'Len of dataset:{len(text)}')

Len of dataset:1115394


In [4]:
# See what are first 500 characters
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [6]:
#get all the unique charater in out dataset
character = sorted(list(set(text)))
vocabsiz = len(character)
print(''.join(character))
print(vocabsiz)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [11]:
#map characters to integers
stoi = {ch : i for i,ch in enumerate(character)}
itos = {i : ch for i,ch in enumerate(character)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[c] for c in l])

test = encode('How are you')
print(test)
print(decode(test))
#other practise is using sentence peice where subwords are encoded not individual characters titoken is used by gpt model

[20, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
How are you


In [12]:
import torch as t
data = t.tensor(encode(text) , dtype=t.long)
print(data.shape,data.dtype)
print(data[:30])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43])


In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

In [14]:
batch_size= 8
train_data[:batch_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:batch_size]
y = train_data[1:batch_size+1]
for i in range(batch_size):
  context = x[:i+1]
  target = y[i]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [16]:
t.manual_seed(1337)
batchsize = 16
blocksize = 32

device = 'cuda' if t.cuda.is_available() else 'cpu'

def get_batch(split):
  data = train_data if split == 'train' else test_data
  ix = t.randint(len(data) - blocksize, (batchsize,))
  x = t.stack([data[i : i+blocksize] for i in ix])
  y = t.stack([data[i+1 : i + blocksize+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

xb , yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('target')
print(yb.shape)
print(yb)

for b in range(batchsize):
  for bl in range(blocksize):
    context = xb[b , :bl + 1]
    target = yb[b , bl]
    print(f"Inputs : {context.tolist()} , Target : {target}")

inputs
torch.Size([16, 32])
tensor([[58, 53,  1, 41, 53, 56, 56, 59, 54, 58,  1, 39,  1, 51, 39, 52,  5, 57,
          1, 61, 47, 44, 43,  1, 47, 57,  0, 61, 46, 43, 52,  1],
        [49,  1, 39, 52,  1, 53, 39, 58, 46,  1, 40, 63,  1, 20, 47, 51,  6,  0,
         32, 46, 43,  1, 59, 52, 47, 58, 63,  1, 58, 46, 43,  1],
        [59, 50, 42,  1, 58, 46, 53, 59,  1, 61, 43, 56, 58,  1, 57, 53,  1, 58,
         53, 53,  2,  0,  0, 24, 33, 15, 21, 27, 10,  0, 35, 43],
        [ 8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 28, 56, 53, 60, 43,  1,
         47, 58,  6,  1, 20, 43, 52, 56, 63,  6,  1, 39, 52, 42],
        [58,  1, 57, 46, 43,  8,  0,  0, 32, 30, 13, 26, 21, 27, 10,  0, 18, 53,
         56,  1, 61, 46, 39, 58,  1, 56, 43, 39, 57, 53, 52,  6],
        [56, 61, 47, 41, 49,  6,  1, 50, 43, 58,  1, 47, 58,  1, 40, 43, 11,  0,
         18, 53, 56,  1, 47, 52,  1, 58, 46, 63,  1, 57, 46, 53],
        [25, 10,  0, 35, 47, 58, 46, 42, 56, 39, 61,  1, 63, 53, 59,  1, 46, 43,
         

In [17]:
from torch import nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
  def __init__(self, vocabsiz):
    super().__init__()

    self.token_embedding = nn.Embedding(vocabsiz , vocabsiz)

  def forward(self , inputs , targets=None):
    logits = self.token_embedding(inputs)
    if targets == None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(-1)
      loss = F.cross_entropy(logits,targets)
    return logits , loss

  def generate(self , inputs ,max_new_token):
    for i in range(max_new_token):
      logits , loss = self(inputs)
      logits = logits[:,-1,:]
      probs = t.softmax(logits , dim =-1)
      ip_next = t.multinomial(probs , num_samples=1)
      inputs = t.cat((inputs , ip_next) , dim = 1)
    return inputs
model = BigramLanguageModel(vocabsiz)
logit , loss = model(xb, yb)
print(logit.shape)
print(loss) #can be directly calculated by by -ln(1/vocab_size)
print(decode(model.generate(inputs= t.zeros((1,1), dtype = t.long),max_new_token=100)[0].tolist()))

torch.Size([512, 65])
tensor(4.6057, grad_fn=<NllLossBackward0>)

DtQOLwf'RIPD'Hmhs xvLwxat:sbYZwLEHy'pryOqOZm$!s$zheQ,;eMkjm,CQufzLw!iWn.e!zDGxA PsS3zdKY!KzGCeJydKJS


In [ ]:
import torch as t

In [18]:
optimizer = t.optim.AdamW(model.parameters() , lr = 1e-3)

In [19]:
batchsize = 32
for steps in range(10000):
  xb , yb  = get_batch('train')
  logits , loss = model(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4426300525665283


In [20]:
print(decode(model.generate(inputs= t.zeros((1,1), dtype = t.long),max_new_token=100)[0].tolist()))


Mave pirance
RDe hicomyonthar's
PES:
AKEd itClel ouratucenonthioneir thondy, y heltieiengerofo'dsssi


In [21]:
t.manual_seed(42)
a = t.tril(t.ones(3,3))
a = a / t.sum(a,1,keepdim=True)
b = t.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [22]:
t.manual_seed(1337)
B,T,C = 4,8,2
x = t.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [23]:
xbow = t.zeros((B,T,C))
for b in range(B):
    for ts in range(T):
        xprev = x[b,:ts+1] # (t,C)
        xbow[b,ts] = t.mean(xprev, 0)

In [ ]:
wei = t.tril(t.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
xbow2 = xbow2.permute(1, 0, 2)
xbow2 = xbow2.permute(1, 0, 2)
t.allclose(xbow , xbow2)

In [ ]:
tril = t.tril(t.ones(T,T))
wei = t.zeros((T,T))
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei , dim = 1)
xbow3 = wei @ x
xbow3[0] , xbow[0] , xbow2[0]

In [ ]:
# from torch import nn
# from torch.nn import functional as F
# n_embed = 32
device = 'cuda' if t.cuda.is_available() else 'cpu'
# t.manual_seed(1337)
#
# model = BigramLanguageModel()
# logit , loss = model(xb, yb)
# print(logit.shape)
# print(loss) #can be directly calculated by by -ln(1/vocab_size)
# print(decode(model.generate(inputs= t.zeros((1,1), dtype = t.long),max_new_token=100)[0].tolist()))

In [ ]:
t.manual_seed(1337)
B , T , C = 4 , 8 , 32
x = t.randn(B,T,C)
head_sz = 16
query = nn.Linear(C , head_sz , bias= False)
key = nn.Linear(C , head_sz , bias= False)
value = nn.Linear(C , head_sz , bias= False)
q = query(x)
k = key(x)
wei = q @ k.transpose(-2,-1)
tril = t.tril(t.ones(T,T))
wei = wei.masked_fill(tril==0 , float('-inf'))
wei = t.softmax(wei , dim = -1)
v = value(x)
out = wei @ v
out.shape

In [15]:
import torch as t
import torch.nn as nn
from torch.nn import functional as F

batch_size = 16
block_size = 32
epochs = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if t.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 64
n_head = 4
n_layer = 4
dropout = 0.1
# ------------

t.manual_seed(1337)

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Train and test splits
data = t.tensor(encode(text), dtype=t.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = t.randint(len(data) - block_size, (batch_size,))
    x = t.stack([data[i:i+block_size] for i in ix])
    y = t.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@t.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = t.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', t.tril(t.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = t.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embed)
        self.linear1 = nn.LayerNorm(n_embed)
        self.linear2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.linear1(x))
        x = x + self.ffwd(self.linear2(x))
        return x

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.norm = nn.LayerNorm(n_embed)
        self.linear = nn.Linear(n_embed, vocab_size)

    def forward(self, inputs, targets=None):
        B, T = inputs.shape
        tok_emb = self.token_embedding_table(inputs)
        pos_emb = self.position_embedding_table(t.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.norm(x)
        logits = self.linear(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = inputs[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            ip_next = t.multinomial(probs, num_samples=1)
            inputs = t.cat((inputs, ip_next), dim=1)
        return inputs

model = BigramLanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
optimizer = t.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    if epoch % eval_interval == 0 or epoch == epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = t.zeros((1, 1), dtype=t.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6700, val loss 2.6801
step 200: train loss 2.5187, val loss 2.5149
step 300: train loss 2.4356, val loss 2.4496
step 400: train loss 2.3678, val loss 2.3744
step 500: train loss 2.3209, val loss 2.3351
step 600: train loss 2.2706, val loss 2.2741
step 700: train loss 2.2355, val loss 2.2479
step 800: train loss 2.1959, val loss 2.2143
step 900: train loss 2.1584, val loss 2.1757
step 1000: train loss 2.1364, val loss 2.1479
step 1100: train loss 2.1047, val loss 2.1401
step 1200: train loss 2.0847, val loss 2.1104
step 1300: train loss 2.0726, val loss 2.0990
step 1400: train loss 2.0385, val loss 2.0675
step 1500: train loss 2.0124, val loss 2.0590
step 1600: train loss 1.9992, val loss 2.0633
step 1700: train loss 1.9898, val loss 2.0423
step 1800: train loss 1.9630, val loss 2.0348
step 1900: train loss 1.9533, val loss 2.0123
step 2000: train loss 1.9384, val loss 2.0184
step 2100: train loss 1.